<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/tool_and_routing_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental"

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [9]:
# Define tools by decorator and using pandatic class as args
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import tool

class SearchInput(BaseModel):
  query: str = Field(description="thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
  """Search for weather online"""
  return "42f"
search.args

{'query': {'description': 'search query', 'title': 'Query', 'type': 'string'}}

In [59]:
# Define the first tool to query current temperature for a position
import requests
class Position(BaseModel):
  latitude: float = Field(description="latitude of the location")
  longitude: float = Field(description="longitude of the location")

@tool(args_schema=Position)
def get_current_temperature(latitude: float, longitude: float) -> str:
  """Get the current temperature"""

  BASE_URL = "https://api.open-meteo.com/v1/forecast"
  params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "temperature_2m",
    "forecast_days": 1,
  }
  response = requests.get(BASE_URL, params=params)
  if response.status_code == 200:
    data = response.json()
    hourly_data = data["hourly"]
    temperature_2m = hourly_data["temperature_2m"]
    unit = data["hourly_units"]["temperature_2m"]

    return f"The current temperature is: {temperature_2m[0]}{unit}"
  else:
    raise Exception(f"Request failed with status code {response.status_code}")


get_current_temperature.invoke({"latitude": 13, "longitude":14})

'The current temperature is: 24.4°C'

In [60]:
from langchain.tools.render import format_tool_to_openai_function
get_current_temperature_function = format_tool_to_openai_function(get_current_temperature)
get_current_temperature_function


{'name': 'get_current_temperature',
 'description': 'Get the current temperature',
 'parameters': {'properties': {'latitude': {'description': 'latitude of the location',
    'type': 'number'},
   'longitude': {'description': 'longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [61]:
# Define prompt and bind function to llm
template_string = "What is the current temperature at position latitude: {latitude}, longitude: {longitude}?"
prompt = PromptTemplate(
    input_variables = ["latitude", "longitude"],
    template=template_string
)
#message_test = prompt.format(latitude=13, longitude=14)
#message_test

llm_with_function = llm.bind(functions = [get_current_temperature_function])

chain = prompt | llm_with_function
res = chain.invoke({"latitude": 13, "longitude":14})
print(res)

content='' additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude": 13.0, "longitude": 14.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--3a5931bc-ce93-45c1-ba9d-663f88892a47-0' tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 13.0, 'longitude': 14.0}, 'id': '8b8e9bd9-442a-49d8-88cf-ff3bf10c2ba6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 75, 'output_tokens': 24, 'total_tokens': 177, 'input_token_details': {'cache_read': 0}}


In [ ]:
!pip install wikipedia

In [67]:
# Define the second tool to use wikipedia
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
  """Search wikipedia for the given query and get page summaries"""
  page_titles = wikipedia.search(query)
  summaries = []
  for page_title in page_titles[:3]:
    try:
      wiki_pedia_page = wikipedia.page(page_title)
      summaries.append(f"Page: {page_title}\nSummary: {wiki_pedia_page.summary}")
    except wikipedia.exceptions.DisambiguationError as e:
      print(f"Skipping disambiguation page: {page_title}")

  if not summaries:
    return "No good Wikipedia Search Result was found"

  return "\n\n".join(summaries)

search_wikipedia_function = format_tool_to_openai_function(search_wikipedia)

In [77]:
# bind model with multiple functions
llm_multi_function = llm.bind(functions = [get_current_temperature_function, search_wikipedia_function])
res1 = llm_multi_function.invoke("What is the current weather at the position: latitude 13, longitide 14 right now?")
res2 = llm_multi_function.invoke("Give a short introduction about the game Werewolves Kill")
print(res1)
print(res2)


content='' additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude": 13.0, "longitude": 14.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--b5c68c90-c565-4ccd-bdb7-4faca86fd95c-0' tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 13.0, 'longitude': 14.0}, 'id': 'dd9d1df3-3000-4a6d-bcb7-afb7d3520e0b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 121, 'output_tokens': 24, 'total_tokens': 233, 'input_token_details': {'cache_read': 0}}
content='' additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query": "Werewolves Kill game"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--cb14cd93-b04b-420f-a10c-54723353c53e-0' tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Werewolves Kill game'}, 'id': 'cbe1ef

In [117]:
# Define the executor as a rounter to trigger tools or return the content
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import BaseMessage

def tool_executor(message: BaseMessage):
    if message.tool_calls:
        tool_call = message.tool_calls[0]
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tools = {
          "get_current_temperature": get_current_temperature,
          "search_wikipedia": search_wikipedia
        }

        # Assuming get_current_temperature is accessible in the scope
        if tool_name in tools:
            return tools[tool_name].invoke(tool_args)
        else:
            # Handle other tools if necessary, or raise an error
            return f"Unknown tool: {tool_name}"
    else:
        return message.content # Return the LLM's content if no tool call

tool_execution_runnable = RunnableLambda(tool_executor)

print("Defined tool_executor function and tool_execution_runnable.")

Defined tool_executor function and tool_execution_runnable.


In [119]:
# Add tool executor to the chain to trigger the function automatically
# from langchain.agents.output_parsers import JsonOutputFunctionsParser
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")]
)

full_chain = prompt | llm_multi_function | tool_execution_runnable

'The current temperature is: 24.4°C'

In [ ]:
full_chain.invoke({"input": "What is the current weather at the position: latitude 13, longitide 14 right now?"})

In [120]:
full_chain.invoke({"input": "Hi, I'm Yx"})

'Hello Yx! Nice to meet you. How can I help you today?'

In [121]:
full_chain.invoke({"input": "What is the game of Werewolves Kill?"})

'Page: Mafia (party game)\nSummary: Mafia, also known as Werewolf, is a social deduction game created in 1986 by Dimitry Davidoff, then a psychology student at Moscow State University. The game models a conflict between two groups: an informed minority (the mafiosi or the werewolves) and an uninformed majority (the villagers). At the start of the game, each player is secretly assigned a role affiliated with one of these teams. The game has two alternating phases: first, a night-phase, during which those with night-killing-powers may covertly kill other players, and second, a day-phase, in which all surviving players debate and vote to eliminate a suspect. The game continues until a faction achieves its win condition; for the village, this usually means eliminating the evil minority, while for the minority, this usually means reaching numerical parity with the village and eliminating any rival evil groups.\n\n\n\nPage: Werewolves Within (film)\nSummary: Werewolves Within is a 2021 Ameri